In [46]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [47]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [48]:
image_asset_path = "images/"

In [49]:

# Import the glob module to retrieve images and annotations
path = glob(image_asset_path+"*.xml")

# Create a dictionary to store the information
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])

# Iterate over all the xml files
for filename in path:
    # Parse the xml file
    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')

    # Retrieve the coordinates
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    # Add the coordinates to the dictionary
    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

# print first 5 rows of the dictionary
pd.DataFrame(labels_dict).head()

,filepath,xmin,xmax,ymin,ymax
0,images\N1.xml,1093,1396,645,727
1,images\N100.xml,134,301,312,350
2,images\N101.xml,31,139,128,161
3,images\N102.xml,164,316,216,243
4,images\N103.xml,813,1067,665,724


In [50]:
# Create CSV file with labels from XML files
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,images\N1.xml,1093,1396,645,727
1,images\N100.xml,134,301,312,350
2,images\N101.xml,31,139,128,161
3,images\N102.xml,164,316,216,243
4,images\N103.xml,813,1067,665,724


In [51]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join(image_asset_path,filename_image)
    return filepath_image
getFilename(filename)

'images/N1.jpeg'

In [63]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['images/N1.jpeg',
 'images/N100.jpeg',
 'images/N101.jpeg',
 'images/N102.jpeg',
 'images/N103.jpeg',
 'images/N104.jpeg',
 'images/N105.jpeg',
 'images/N106.jpeg',
 'images/N107.jpeg',
 'images/N108.jpeg']

In [75]:
file_path = image_path[0] #path of our image N1.jpeg
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=1000, height=1000, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=1093, x1=1396, y0=645, y1=727, xref='x', yref='y',line_color='cyan')

### Convert each and every image into an array using OpenCV and resize the image into 224 x 224 which is the standard compatible size of the pre-trained transfer learning model.

In [78]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

data[0]

array([[[0.8352941 , 0.9764706 , 0.96862745],
        [0.8392157 , 1.        , 0.99607843],
        [0.8392157 , 1.        , 0.99607843],
        ...,
        [0.5764706 , 0.44313726, 0.4392157 ],
        [0.57254905, 0.4509804 , 0.44313726],
        [0.6431373 , 0.5254902 , 0.5176471 ]],

       [[0.9137255 , 1.        , 1.        ],
        [0.88235295, 1.        , 1.        ],
        [0.87058824, 0.99607843, 0.99215686],
        ...,
        [0.58431375, 0.44313726, 0.44313726],
        [0.52156866, 0.3882353 , 0.39215687],
        [0.54901963, 0.41568628, 0.41960785]],

       [[0.9019608 , 0.9882353 , 0.9764706 ],
        [0.8980392 , 1.        , 0.99607843],
        [0.88235295, 1.        , 1.        ],
        ...,
        [0.5529412 , 0.39607844, 0.4       ],
        [0.5647059 , 0.4117647 , 0.42352942],
        [0.5411765 , 0.4       , 0.40784314]],

       ...,

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 0.99215686],
        [0.9882353 , 0